In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
def conv3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion=1
    def __init__(self, in_planes, planes, stride=1, downsample=None, is_last=False):
        super(BasicBlock, self).__init__()
        self.is_last = is_last
        self.conv1 = conv3(in_planes, planes, stride) # 들어가는 차원의 크기 in_planes 에서 나가는 planes
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=False) # inplace=False 원본 데이터 보존하는 대신 추가메모리 사용,
        # 만약, inplace=True라면 들어가는 인수 또한 값이 output과 동일하게 바뀌는 현상 발생
        self.conv2 = conv3(planes, planes) # 들낙하는 차원은 planes로 fix
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        # downsample이 왜 필요한가? : 입력 텐서와 출력 텐서의 차원 불일치 문제 해소
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        # out은 잔차와 더해져서 다음 out은 relu를 거친것이고, preact는 relu 거치기전(pre-relu)
        preact = out.clone() # out.clone()은 무슨 의미인가? copy()와 같은 의미인가? Pytorch에서 텐서 복사할때 clone 메서드 사용
        out = self.relu(out)
        
        if self.is_last:
            # is_last가 각 블록마다의 distillation해야할 feature를 내보내야할 시기인가?
            return out, preact
        else:
            return out
    
class ResNet(nn.Module):
    def __init__(self, depth, num_classes=10):
        super(ResNet, self).__init__()
        
        # assert 가 경고한다는건데, ResNet-20, 32, 44 이런식으로 맞추는게 필요한가?
        assert (depth - 2) % 6 == 0, 'depth should be one of 20, 32, 44, 56, 110'
        block_num = (depth - 2) // 6 # block_num??
        self.in_planes = 16
        self.conv1 = conv3(in_planes=3, out_planes=16)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=False)
        self.layer1 = self._make_layer(planes=16, block_num=block_num)
        self.layer2 = self._make_layer(planes=32, block_num=block_num, stride=2)
        self.layer3 = self._make_layer(planes=64, block_num=block_num, stride=2)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
        for m in self.modules(): # ResNet 모델의 모든 Conv층, BN층을 접근해서 가중치 초기화 수행
            if isinstance(m, nn.Conv2d):
                # Conv층 같은 경우, fan_out(출력 유닛에 초점) <-> fan_in(입력 유닛에 초점) 을 사용
                # 그리고 ReLU 활성화함수를 거침에 따라 nonlinearity를 'relu'로 설정
                # kaiming_normal_ == He초기화
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            
    
    def _make_layer(self, planes, block_num, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            # stride가 1이 아니거나 in_planes와 planes가 같지 않다면 downsample 변동
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(BasicBlock(self.in_planes, planes, stride, downsample))
        self.in_planes = planes
        for i in range(1, block_num):
            layers.append(BasicBlock(self.in_planes, planes))
        return nn.Sequential(*layers) 
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = F.relu(x) ## nn.ReLU() 은 모듈 객체를 생성함에 따라 __init__에서 주로 쓰이고, F.relu는 모듈 객체를 생성하지 않으므로 forward에서 자주 쓰임
        x = self.avgpool(x)
        x = x.view(x.size(0), -1) # == x.flatten(start_dim=1), 1차원으로 평탄화, view 쓰는 경우에는 복잡한 차원 재구성이 필요할때
        x = self.fc(x)
        return x
    
    def get_bn_before_relu(self):
        
        if isinstance(self.layer1[0], BasicBlock):
            bn1 = self.layer1[-1].bn2
            bn2 = self.layer2[-1].bn2
            bn3 = self.layer3[-1].bn2
        else:
            print('ResNet unknown block error')
        
        return [bn1, bn2, bn3]
    
    def get_channel_num(self):
        
        return [16, 32, 64]
    
    def extract_feature(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        
        feat1 = self.layer1(x)
        feat2 = self.layer2(feat1)
        feat3 = self.layer3(feat2)
        
        x = nn.ReLU(inplace=False)(feat3)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        out = self.fc(x)
        
        return [feat1, feat2, feat3], out
    
def resnet20(class_num=10):
    return ResNet(20, class_num)
    
def resnet32(class_num=10):
    return ResNet(32, class_num)
    
def resnet44(class_num=10):
    return ResNet(44, class_num)

In [8]:
t_net = resnet20
t_net

<function __main__.resnet20(class_num=10)>